# Tolerance analysis of simple 1.5D model, automatic dictionary construction, crude monte carlo, optimizaiton on epistemic space of uncertainty.

In [1]:
import os
import pickle
import re
import pprint
import numpy as np
import sympy as sp
import scipy
import openturns as ot
import matplotlib.pyplot as plt
import trimesh as tr

from math import pi, sqrt
from joblib import Parallel, delayed
from importlib import reload
from IPython.display import display, clear_output, HTML, IFrame
from time import time, sleep
from sympy.printing import latex
from trimesh import viewer as trview
from scipy.optimize import OptimizeResult, minimize, Bounds, LinearConstraint, shgo, basinhopping, direct, dual_annealing

import otaf
#from efficient_kan import KAN, KANLinear

notebook_name = os.path.splitext(os.path.basename(os.environ.get("JPY_SESSION_NAME")))[0]

In [2]:
### Different measures of our problem
X1 = 99.8   # Nominal Length of the male piece
X2 = 100.0  # Nominal Length of the female piece
X3 = 10.0   # Nominal height of the pieces
t = 0.2*sqrt(2)    # Tolerance for X1 and X2. (95% conform)  (= t/2)

## Coordinates, points, feature definitions.

In [3]:
# Global coordinate system
R0 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
x_, y_, z_ = R0[0], R0[1], R0[2]

# Important points
# Pièce 1 (male)
P1A0, P1A1, P1A2 = (
    np.array((0, X3 / 2, 0.0)),
    np.array((0, X3, 0.0)),
    np.array((0, 0, 0.0)),
)
P1B0, P1B1, P1B2 = (
    np.array((X1, X3 / 2, 0.0)),
    np.array((X1, X3, 0.0)),
    np.array((X1, 0, 0.0)),
)
P1C0, P1C1, P1C2 = (
    np.array((X1 / 2, 0, 0.0)),
    np.array((0, 0, 0.0)),
    np.array((X1, 0, 0.0)),
)

# Pièce 2 (femelle)  # On met les points à hM et pas hF pour qu'ils soient bien opposées! (Besoin??)
P2A0, P2A1, P2A2 = (
    np.array((0, X3 / 2, 0.0)),
    np.array((0, X3, 0.0)),
    np.array((0, 0, 0.0)),
)
P2B0, P2B1, P2B2 = (
    np.array((X2, X3 / 2, 0.0)),
    np.array((X2, X3, 0.0)),
    np.array((X2, 0, 0.0)),
)
P2C0, P2C1, P2C2 = (
    np.array((X2 / 2, 0, 0.0)),
    np.array((0, 0, 0.0)),
    np.array((X2, 0, 0.0)),
)

# Local coordinate systems
# Pièce1
RP1a = np.array([-1 * x_, -1 * y_, z_])
RP1b = R0
RP1c = np.array([-y_, x_, z_])

# Pièce2
RP2a = R0
RP2b = np.array([-1 * x_, -1 * y_, z_])
RP2c = np.array([y_, -1 * x_, z_])

### Construction of the augmented system data dictionary.

In [4]:
system_data = {
    "PARTS" : {
        '1' : {
            "a" : {
                "FRAME": RP1a,
                "POINTS": {'A0' : P1A0, 'A1' : P1A1, 'A2' : P1A2},
                "TYPE": "plane",
                "INTERACTIONS": ['P2a'],
                "CONSTRAINTS_D": ["PERFECT"], # In this modelization, only defects on the right side
                "CONSTRAINTS_G": ["FLOATING"],            
            },
            "b" : {
                "FRAME": RP1b,
                "POINTS": {'B0' : P1B0, 'B1' : P1B1, 'B2' : P1B2},
                "TYPE": "plane",
                "INTERACTIONS": ['P2b'],
                "CONSTRAINTS_D": ["NONE"],
                "CONSTRAINTS_G": ["FLOATING"],            
            },
            "c" : {
                "FRAME": RP1c,
                "POINTS": {'C0' : P1C0, 'C1' : P1C1, 'C2' : P1C2},
                "TYPE": "plane",
                "INTERACTIONS": ['P2c'],
                "CONSTRAINTS_D": ["PERFECT"],
                "CONSTRAINTS_G": ["SLIDING"],            
            },
        },
        '2' : {
            "a" : {
                "FRAME": RP2a,
                "POINTS": {'A0' : P2A0, 'A1' : P2A1, 'A2' : P2A2},
                "TYPE": "plane",
                "INTERACTIONS": ['P1a'],
                "CONSTRAINTS_D": ["PERFECT"], # In this modelization, only defects on the right side
                "CONSTRAINTS_G": ["FLOATING"],            
            },
            "b" : {
                "FRAME": RP2b,
                "POINTS": {'B0' : P2B0, 'B1' : P2B1, 'B2' : P2B2},
                "TYPE": "plane",
                "INTERACTIONS": ['P1b'],
                "CONSTRAINTS_D": ["NONE"],
                "CONSTRAINTS_G": ["FLOATING"],            
            },
            "c" : {
                "FRAME": RP2c,
                "POINTS": {'C0' : P2C0, 'C1' : P2C1, 'C2' : P2C2},
                "TYPE": "plane",
                "INTERACTIONS": ['P1c'],
                "CONSTRAINTS_D": ["PERFECT"],
                "CONSTRAINTS_G": ["SLIDING"],            
            },
        }  
    },
    "LOOPS": {
        "COMPATIBILITY": {
            "L0": "P1cC0 -> P2cC0 -> P2aA0 -> P1aA0",
            "L1": "P1cC0 -> P2cC0 -> P2bB0 -> P1bB0",
        },
    },
    "GLOBAL_CONSTRAINTS": "2D_NZ",
}

In [5]:
SDA = otaf.AssemblyDataProcessor(system_data)
SDA.generate_expanded_loops()

In [6]:
CLH = otaf.CompatibilityLoopHandling(SDA)
compatibility_expressions = CLH.get_compatibility_expression_from_FO_matrices()

In [7]:
ILH = otaf.InterfaceLoopHandling(SDA, CLH, circle_resolution=20)
interface_constraints = ILH.get_interface_loop_expressions()

Processing part 2, surface a for plane-to-plane interactions.
usedGMatDat [['2', 'a', 'A0', '1', 'a', 'A0']]
Found 1 used gap matrices.
unusedGMatDat [['2', 'a', 'A2', '1', 'a', 'A2'], ['2', 'a', 'A1', '1', 'a', 'A1']]
Found 2 unused gap matrices.
Generated 2 interaction matrix loops for current matching.
Processing part 2, surface b for plane-to-plane interactions.
usedGMatDat [['2', 'b', 'B0', '1', 'b', 'B0']]
Found 1 used gap matrices.
unusedGMatDat [['2', 'b', 'B1', '1', 'b', 'B1'], ['2', 'b', 'B2', '1', 'b', 'B2']]
Found 2 unused gap matrices.
Generated 2 interaction matrix loops for current matching.


In [8]:
SOCAM = otaf.SystemOfConstraintsAssemblyModel(
    compatibility_expressions, interface_constraints
)

SOCAM.embedOptimizationVariable()

print(len(SOCAM.deviation_symbols), SOCAM.deviation_symbols)

4 [u_d_4, gamma_d_4, u_d_5, gamma_d_5]


## Construction of the stochastic model of the defects. (old lambda approach)

In [9]:
Cm = 1.0
sigma_e_pos = t / (6 * Cm)

# Le défaut en orientation est piloté par une incertitude sur un angle. On suppose les angles petits << 1 rad
theta_max = t / X3
sigma_e_theta = (2*theta_max) / (6*Cm) 

In [10]:
RandDeviationVect = otaf.distribution.get_composed_normal_defect_distribution(
    defect_names=SOCAM.deviation_symbols,
    sigma_dict = {"alpha":sigma_e_theta, 
                  "beta":sigma_e_theta,
                  "gamma":sigma_e_theta, 
                  "u":sigma_e_pos, 
                  "v":sigma_e_pos, 
                  "w":sigma_e_pos})

cons, linearConstraint = otaf.optimization.lambda_constraint_dict_from_composed_distribution(RandDeviationVect)
bounds_lambda = otaf.optimization.bounds_from_composed_distribution(RandDeviationVect)

## Construction of a neural network based surrogate 
(could be omitted but makes things faster)

#### First generate the training sample :

In [11]:
# Define the seed, sample size, and file paths
SEED = 420  # Example seed value
sample_size = 100000
model_name = notebook_name
sample_filename = f'STORAGE/training_sample_{sample_size}_seed_{SEED}_{model_name}_ai.npy'
results_filename = f'STORAGE/training_results_{sample_size}_seed_{SEED}_{model_name}_ai.npy'

# Ensure reproducibility by setting the seed
np.random.seed(SEED)

# Check if the sample and results files already exist
if os.path.exists(sample_filename) and os.path.exists(results_filename):
    with open(sample_filename, 'rb') as file:
        TRAIN_SAMPLE = np.load(file)
    with open(results_filename, 'rb') as file:
        TRAIN_RESULTS = np.load(file)
    print("Loaded existing sample and results from file.")
else:
    # Generate the sample
    dist = otaf.distribution.multiply_composed_distribution_with_constant(
        RandDeviationVect, 1.15) # We now work with low failure probabilities
    #TRAIN_SAMPLE = np.array(otaf.uncertainty.generateLHSExperiment(dist, sample_size))
    TRAIN_SAMPLE = np.array(dist.getSample(sample_size),dtype="float32")
    # Compute the results
    TRAIN_RESULTS = otaf.uncertainty.compute_gap_optimizations_on_sample_batch(
        SOCAM,
        TRAIN_SAMPLE,
        bounds=None,
        n_cpu=-2,
        progress_bar=True,
        batch_size=500,
        dtype="float32"
    )
    #TRAIN_RESULTS = np.array([res.x[-1] for res in TRAIN_RESULTS],dtype="float32") #Only s variable.
    
    # Save the sample and results
    with open(sample_filename, 'wb') as file:
        np.save(file, TRAIN_SAMPLE)
    with open(results_filename, 'wb') as file:
        np.save(file, TRAIN_RESULTS)
    print("Generated and saved new sample and results with seed.")

# Assign X and y from TRAIN_SAMPLE and TRAIN_RESULTS
Xtrain = TRAIN_SAMPLE
ytrain = TRAIN_RESULTS
print(f"Ratio of failed simulations in sample : {np.where(ytrain[:,-1]<0,1,0).sum()/sample_size}")

  0%|          | 0/100000 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'STORAGE/training_sample_100000_seed_420_Modele1.5D_Auto_Optim_CMC_ai.npy'

#### Then train the NN model.

In [ ]:
save_path = f'STORAGE/{notebook_name}.pth'
load = False
dim = int(RandDeviationVect.getDimension())
neural_model = otaf.surrogate.NeuralRegressorNetwork(
    dim, 1,
    Xtrain, ytrain[:,-1], 
    clamping=True, 
    finish_critertion_epoch=5,
    loss_finish=1e-6, 
    metric_finish=0.99999, 
    max_epochs=500, 
    batch_size=30000, 
    compile_model=False, 
    train_size=0.6, 
    save_path = save_path,
    input_description=RandDeviationVect.getDescription(),
    display_progress_disable=False)

lr=0.003

#neural_model.model = KAN([dim, 8, 4, 1])  #otaf.surrogate.get_base_relu_mlp_model(dim, 1, False)

neural_model.model = otaf.torch.nn.Sequential(
    *otaf.surrogate.get_custom_mlp_layers([dim, 100, 70, 30, 1], activation_class=otaf.torch.nn.GELU)
)

neural_model.optimizer = otaf.torch.optim.AdamW(neural_model.parameters(), lr=lr, weight_decay=1e-4)
otaf.surrogate.initialize_model_weights(neural_model)
neural_model.scheduler =  otaf.torch.optim.lr_scheduler.ExponentialLR(neural_model.optimizer, 1.0001)
neural_model.loss_fn = otaf.torch.nn.MSELoss()
#neural_model.loss_fn = otaf.uncertainty.LimitSpaceFocusedLoss(0.0001, 2, square=True) # otaf.uncertainty.PositiveLimitSpaceFocusedLoss(0.0001, 2, 4, square=False)

if os.path.exists(save_path) and load:
    neural_model.load_model()
else :
    neural_model.train_model()
    neural_model.plot_results()
    neural_model.save_model()

## Optimization on the imprecise space of defects, to get upper and lower probability of failure given the constraints on the defect parameters.

In [ ]:
# Threshold and scaling factors
scale_factor = 1e6  # Adjust this scaling factor for your specific range


N_SAMPLE_MINI = int(1e7)
standards = [RandDeviationVect.getParameter()[i] for i , param in enumerate(RandDeviationVect.getParameterDescription()) if "sigma" in param] 
means = [RandDeviationVect.getParameter()[i] for i , param in enumerate(RandDeviationVect.getParameterDescription()) if "mu" in param] 
sample = np.array(RandDeviationVect.getSample(N_SAMPLE_MINI))
threshold = 0

def model(x):
    # Direct model without ai
    gap_variable_array = otaf.uncertainty.compute_gap_optimizations_on_sample_batch(
        SOCAM, x, n_cpu=-1, progress_bar=True
    )
    slack_variable = gap_variable_array[:, -1]
    return slack_variable

def model2(x): 
    # Surrogate ai model
    return np.squeeze(neural_model.evaluate_model_non_standard_space(x).detach().numpy())

def optimization_function_mini(x, getJac=True, model=model2, scale_factor=scale_factor): 
    if getJac:
        res = otaf.uncertainty.monte_carlo_non_compliancy_rate_w_gradient(
            threshold, sample, means, standards, model, model_is_bool=True)(x)
        # Scale both objective and Jacobian
        return res[0] * scale_factor, res[1] * scale_factor
    else:
        x = sample * np.sqrt(x[np.newaxis, :])
        return model(x).mean() * scale_factor  # Scale mean value

def optimization_function_maxi(x, getJac=True, model=model2, scale_factor=scale_factor): 
    if getJac:
        res = otaf.uncertainty.monte_carlo_non_compliancy_rate_w_gradient(
            threshold, sample, means, standards, model, model_is_bool=True)(x)
        # Scale both objective and Jacobian, but remember this is maximization, so we multiply by -1
        return -1 * res[0] * scale_factor, -1 * res[1] * scale_factor
    else:
        x = sample * np.sqrt(x[np.newaxis, :])
        return -1 * model(x).mean() * scale_factor  # Scale mean value

# Define the callback function
def print_callback(xk):
    print(f"Current parameter values: {xk}")

In [ ]:
res = minimize(optimization_function_maxi,[0.3, 0.7]*2, 
         jac=True, method="SLSQP", args=(True,),
         options={"disp":True, "maxiter":100, "ftol":1e-6, 'eps':0.1},      
         bounds=bounds_lambda, constraints=cons, callback=print_callback)

display(res)

In [ ]:
res = minimize(optimization_function_mini,[1, 1, 1, 1], 
         jac=False, method="SLSQP", args=(False,),
         options={"disp":True, "maxiter":100, "ftol":1e-8, "eps":0.1},      
         bounds=bounds_lambda, constraints=cons, callback=print_callback)

display(res)

#### As you can see, the optimization is not capable of finding the minimum/maximum, due to the middle zone being flat. 

## Let's try global optimization !

In [ ]:
# Basinhopping for the maximization function
x0_maxi = [.5]*4  # Initial guess

minimizer_kwargs_maxi = {
    "method": "SLSQP",
    "args": (False,),
    "constraints": cons,
    "bounds": bounds_lambda,
    "options": {"disp": False, "maxiter": 100, "ftol": 1e-6, "eps":0.01}
}

res_maxi = basinhopping(optimization_function_maxi, x0_maxi, 
                        niter=80, 
                        T=5.5, 
                        stepsize=2.3, 
                        niter_success=19,
                        interval=15,
                        target_accept_rate=0.44,
                        stepwise_factor=0.73,                        
                        minimizer_kwargs=minimizer_kwargs_maxi, disp=True)


print("Maximization Result:")
print(res_maxi)

In [ ]:
# Basinhopping for the minimization function
x0_mini = [.5]*4  # Initial guess

minimizer_kwargs_mini = {
    "method": "SLSQP",
    "args": (False,),
    "constraints": cons,
    "bounds": bounds_lambda,
    "options": {"disp": True, "maxiter": 100, "ftol": 1e-8, "eps":0.001}
}

res_mini = basinhopping(optimization_function_mini, x0_mini,
                        niter=80, 
                        T=5.5, 
                        stepsize=2.3, 
                        niter_success=19,
                        interval=15,
                        target_accept_rate=0.44,
                        stepwise_factor=0.73,                        
                        minimizer_kwargs=minimizer_kwargs_maxi, disp=True)

print("Minimization Result:")
print(res_mini)

best_params_maxi

best_params_mini

### Results :

In [ ]:
results_array = np.array([[80.182,  1.276,  3.110, 19.326, 85.911, 0.445,  0.726],
[55.866,  1.695,  1.252, 97.917, 11.467, 0.510 ,  0.580],
[81.046,  7.024,  3.161, 52.917 , 22.954, 0.122,  0.564],
[57.059,  7.767,  2.544, 64.469, 16.478, 0.424,  0.887],
[77.686  ,  4.737,  1.854, 88.378, 13.230, 0.223,  0.892],
[177.175,   5.441,   4.900,  82.017 , 15.701,   0.710,   0.777],
[69.056,  3.553,  2.071, 29.702, 26.634 , 0.324,  0.957],
[100.007,   5.310,   1.037,  96.541, 20.207 ,   0.835,   0.982],
[55.337,  5.494,  0.848, 39.742, 23.283 , 0.279 ,  0.731],
[122.673,   4.215 ,   3.807,  38.694, 15.465,   0.339,   0.606],
[95.877,  5.579,  0.654, 66.933, 25.929, 0.406,  0.513],
[167.494,   8.662,   2.578 ,  63.197, 12.251,   0.830,   0.806],
[50.299,  6.270,  0.807, 53.847, 66.134, 0.449,  0.690],
[176.291,   5.938,   3.781,  52.256, 16.557,   0.728,   0.513]])

In [ ]:
data = results_array
# Extract bounds for normalization
bounds = np.array(list(param_bounds.values()))
min_bounds = bounds[:, 0]
max_bounds = bounds[:, 1]

# Normalize the data
normalized_data = (data - min_bounds) / (max_bounds - min_bounds)

# Calculate the median and IQR
median_normalized = np.median(normalized_data, axis=0)
q1_normalized = np.percentile(normalized_data, 25, axis=0)
q3_normalized = np.percentile(normalized_data, 75, axis=0)

# Create the plot
fig, ax = plt.subplots(figsize=(12, 8))

for i, row in enumerate(normalized_data):
    ax.plot(range(1, len(row) + 1), row, marker='o', label=f'Trajectory {i + 1}')

# Plot the median
ax.plot(range(1, len(median_normalized) + 1), median_normalized, color='black', linewidth=2, label='Median')

# Plot the IQR as a see-through gray area
ax.fill_between(range(1, len(median_normalized) + 1),
                q1_normalized,
                q3_normalized,
                color='gray', alpha=0.3, label='Interquartile Range')

# Set plot labels and title
ax.set_xlabel('Parameter Index')
ax.set_ylabel('Normalized Value')
ax.set_title('Normalized Parameter Trajectories from LHS Runs')
ax.set_xticks(range(1, len(param_bounds) + 1))
ax.set_xticklabels(param_bounds.keys(), rotation=45)
ax.legend()

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
np.median(data,axis=0)